In [29]:
import sqlite3
import numpy as np
from scrapy import Selector
from datetime import date
import requests
import os.path
import re

In [45]:
def check_database():
    if os.path.isfile("ProductPrices.db"):
        print("DataBase exists")
        connection = sqlite3.connect("ProductPrices.db") 
        crsr = connection.cursor() 
        return connection,crsr
    else:
        print("No previous database exists , new created")
        connection = sqlite3.connect("ProductPrices.db") 
        crsr = connection.cursor()
        create="""CREATE TABLE Prices(
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        Price INTEGER,
        Date VARCHAR
        )"""
        crsr.execute(create)
        connection.commit() 
        return connection,crsr

In [60]:
def get_price(url,connection=connection,crsr=crsr):
    today=date.today()
    check="SELECT * FROM Prices WHERE Date="+"'"+str(today)+"'"+" LIMIT 1"
    crsr.execute(check)
    cond=crsr.fetchall()
    if cond:
        print("price exist")
    else:
        html=requests.get(url).content
        sel=Selector(text=html)
        price=sel.css('div._1vC4OE._3qQ9m1::text').extract()
        price=price[0][1:]
        price=re.sub(',','',price)
        day="'"+str(date.today())+"'"
        add="INSERT INTO Prices(Price , Date) VALUES("+price+","+day+")"
        crsr.execute(add)
        minimum_price="SELECT MIN(Price) FROM PRICES"
        crsr.execute(minimum_price)
        min_price=crsr.fetchall()
        if min_price>int(price):
            print("This is the minimu price o")
        connection.commit() 
        print("price added")    

In [ ]:
print("Enter the Flipkart's product url")
connection,crsr=check_database()
url=input()
get_price(url,connection,crsr)

In [67]:
check="SELECT * FROM Prices"
crsr.execute(check)
cond=crsr.fetchall()
cond

[(1, 28990, '2020-07-11')]

In [63]:
connection.close()